In [1]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
import re
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('Twitter_Data.csv')
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [3]:
df=df.dropna()
df = df.reset_index(drop=True)
df['category'].value_counts()

 1.0    72249
 0.0    55211
-1.0    35509
Name: category, dtype: int64

In [4]:
df=df[df['category']!=0]
df['category'].value_counts()

 1.0    72249
-1.0    35509
Name: category, dtype: int64

In [5]:
df = df.sample(frac=1).reset_index(drop=True) #shuffling of tweets
data=df[df['category']==-1][:35000]
data=data.append(df[df['category']==1][:35000])
data = data.reset_index(drop=True)
display(data['category'].value_counts())
data

C:\Users\CPORGALINAS\AppData\Local\Temp\ipykernel_13792\2233999524.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df[df['category']==1][:35000])


-1.0    35000
 1.0    35000
Name: category, dtype: int64

,clean_text,category
0,chor bhai bhai please invite too join your com...,-1.0
1,read ashley tellis’ article three feats modi t...,-1.0
2,opposition always resorted mockery personal at...,-1.0
3,think should disallow second seats its crimina...,-1.0
4,look the woman security india under the modi g...,-1.0
...,...,...
69995,very rare pic nehru launching missile hit satt...,1.0
69996,got himself hour free diverted nation’ attenti...,1.0
69997,very true the modi era the best thing have hap...,1.0
69998,product placements bollywood films exploded th...,1.0


In [6]:
data['pre_process'] = data['clean_text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['pre_process']=data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(r"http\S+", "", x))

C:\Users\CPORGALINAS\Documents\Machine Learning\Revalida 2\env\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
def contractions(s):
    s = re.sub(r"won't", "will not",s)
    s = re.sub(r"would't", "would not",s)
    s = re.sub(r"could't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
data['pre_process']=data['pre_process'].apply(lambda x:contractions(x))

In [8]:
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

In [10]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [11]:
data

,clean_text,category,pre_process
0,chor bhai bhai please invite too join your com...,-1.0,chor bhai bhai please invite join company wais...
1,read ashley tellis’ article three feats modi t...,-1.0,read ashley tellis article three feat modi lon...
2,opposition always resorted mockery personal at...,-1.0,opposition always resorted mockery personal at...
3,think should disallow second seats its crimina...,-1.0,think disallow second seat criminal waste publ...
4,look the woman security india under the modi g...,-1.0,look woman security india modi government poli...
...,...,...,...
69995,very rare pic nehru launching missile hit satt...,1.0,rare pic nehru launching missile hit sattelite...
69996,got himself hour free diverted nation’ attenti...,1.0,got hour free diverted nation attention addres...
69997,very true the modi era the best thing have hap...,1.0,true modi era best thing happened india
69998,product placements bollywood films exploded th...,1.0,product placement bollywood film exploded turn...


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data['pre_process'], data['category'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (52500,) (52500,) Test:  ((17500,), (17500,))


In [13]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


In [14]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [15]:
clf.fit(tf_x_train,Y_train)

LinearSVC(random_state=0)

In [16]:
y_test_pred=clf.predict(tf_x_test)

In [17]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [18]:
report

{'-1.0': {'precision': 0.8853759356496481,
  'recall': 0.8908498201438849,
  'f1-score': 0.888104443323808,
  'support': 8896},
 '1.0': {'precision': 0.8864194642648263,
  'recall': 0.8807531380753139,
  'f1-score': 0.8835772168133854,
  'support': 8604},
 'accuracy': 0.8858857142857143,
 'macro avg': {'precision': 0.8858976999572372,
  'recall': 0.8858014791095994,
  'f1-score': 0.8858408300685967,
  'support': 17500},
 'weighted avg': {'precision': 0.8858889939470763,
  'recall': 0.8858857142857143,
  'f1-score': 0.8858786000726265,
  'support': 17500}}

In [20]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver='saga')

In [21]:
clf.fit(tf_x_train,Y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [22]:
y_test_pred=clf.predict(tf_x_test)

In [23]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)

In [25]:
report

{'-1.0': {'precision': 0.8813878727413812,
  'recall': 0.8937724820143885,
  'f1-score': 0.8875369760562594,
  'support': 8896},
 '1.0': {'precision': 0.8885481778511617,
  'recall': 0.8756392375639238,
  'f1-score': 0.8820464789556869,
  'support': 8604},
 'accuracy': 0.8848571428571429,
 'macro avg': {'precision': 0.8849680252962715,
  'recall': 0.8847058597891562,
  'f1-score': 0.8847917275059731,
  'support': 17500},
 'weighted avg': {'precision': 0.8849082878936413,
  'recall': 0.8848571428571429,
  'f1-score': 0.8848375339389265,
  'support': 17500}}

{'-1.0': {'precision': 0.8813878727413812, 'recall': 0.8937724820143885, 'f1-score': 0.8875369760562594, 'support': 8896}, '1.0': {'precision': 0.8885481778511617, 'recall': 0.8756392375639238, 'f1-score': 0.8820464789556869, 'support': 8604}, 'accuracy': 0.8848571428571429, 'macro avg': {'precision': 0.8849680252962715, 'recall': 0.8847058597891562, 'f1-score': 0.8847917275059731, 'support': 17500}, 'weighted avg': {'precision': 0.8849082878936413, 'recall': 0.8848571428571429, 'f1-score': 0.8848375339389265, 'support': 17500}}


In [27]:
print(classification_report(Y_test, y_test_pred))

              precision    recall  f1-score   support

        -1.0       0.88      0.89      0.89      8896
         1.0       0.89      0.88      0.88      8604

    accuracy                           0.88     17500
   macro avg       0.88      0.88      0.88     17500
weighted avg       0.88      0.88      0.88     17500

